In [5]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# >> Visualization
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
from scipy.stats import chi2_contingency
from IPython.display import display
from itertools import combinations
from scipy.stats import kstest, spearmanr, pearsonr
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [6]:
data = pd.read_csv("finanzas-hotel-bookings.csv", index_col=0)

In [7]:
data['arrival_date_month'].value_counts()

arrival_date_month
August       13877
July         12661
May          11791
October      11160
April        11089
June         10939
September    10508
November      6794
December      6780
March         5922
February      4898
3             3872
January       3581
2             3170
1             2348
Name: count, dtype: int64

### Eliminar columnas 

In [8]:
# Eliminar múltiples columnas
data = data.drop(['arrival_date_week_number', '0', 'company'], axis=1)

In [9]:
data.shape

(182877, 29)

### Nulos

In [10]:
data.head(1)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,False,342.0,2015.0,July,1.0,0.0,0.0,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00


In [11]:
def variables_categoricas(data):
   
    # Identificar columnas categóricas con valores nulos
    nulos_categoricas = data[data.columns[data.isnull().any()]].select_dtypes(include="O").columns
    print("Las columnas categóricas que tienen nulos son : \n")
    print(nulos_categoricas)

    # Listas de columnas para reemplazar por la moda y por "Unknown", 
    columnas_moda = ['reserved_room_type', 'assigned_room_type']
    columnas_desconocido = ['arrival_date_month','is_repeated_guest','hotel','is_canceled','country','customer_type','reservation_status','arrival_date_month']
    columnas_no_definido = ['meal', 'market_segment','distribution_channel']

    # Reemplazar nulos con la moda 
    for columna in columnas_moda:
        moda = data[columna].mode()[0]  
        data[columna] = data[columna].fillna(moda)  

    # Reemplazar nulos con "Unknown" 
    for columna in columnas_desconocido:
        data[columna] = data[columna].fillna("Unknown")

    # Reemplazar nulos con "Undefined" 
    for columna in columnas_no_definido:
        data[columna] = data[columna].fillna("Undefined")
    
    # Verificar si se han eliminado los nulos en las columnas imputadas con moda
    print("\nDespués del reemplazo usando 'fillna' quedan los siguientes nulos en columnas de moda:")
    print(data[columnas_moda].isnull().sum())
    
    # Verificar si se han eliminado los nulos en las columnas de "Unknown"
    print("Después del reemplazo usando 'fillna' quedan los siguientes nulos en columnas de 'Unknown':")
    print(data[columnas_desconocido].isnull().sum())

    # Verificar si se han eliminado los nulos en las columnas de "Undefined"
    print("\nDespués del reemplazo usando 'fillna' quedan los siguientes nulos en columnas de 'Undefined':")
    print(data[columnas_no_definido].isnull().sum())
    
    return data

# Imputar valores nulos en variables categóricas
df_categoricas = variables_categoricas(data)

Las columnas categóricas que tienen nulos son : 

Index(['hotel', 'is_canceled', 'arrival_date_month', 'meal', 'country',
       'market_segment', 'distribution_channel', 'reserved_room_type',
       'assigned_room_type', 'customer_type', 'reservation_status',
       'reservation_status_date'],
      dtype='object')

Después del reemplazo usando 'fillna' quedan los siguientes nulos en columnas de moda:
reserved_room_type    0
assigned_room_type    0
dtype: int64
Después del reemplazo usando 'fillna' quedan los siguientes nulos en columnas de 'Unknown':
arrival_date_month    0
is_repeated_guest     0
hotel                 0
is_canceled           0
country               0
customer_type         0
reservation_status    0
arrival_date_month    0
dtype: int64

Después del reemplazo usando 'fillna' quedan los siguientes nulos en columnas de 'Undefined':
meal                    0
market_segment          0
distribution_channel    0
dtype: int64


In [12]:
def variables_numericas(data):
   
    # Identificar columnas numéricas con valores nulos
    nulos_numericas = data[data.columns[data.isnull().any()]].select_dtypes(include=np.number).columns
    print("Las columnas numéricas que tienen nulos son : \n")
    print(nulos_numericas)

    # Mostrar el porcentaje de valores nulos 
    print(data[nulos_numericas].isnull().sum() / data.shape[0] * 100)
   
    # Reemplazar nulos con la media 
    media = data[['adr']].mean()
    data[['adr']] = data[['adr']].fillna(media)

    # Verificar que los nulos en 'adr' fueron reemplazados
    print("Después de reemplazar nulos con la media:")
    print(data['adr'].isnull().sum())
    
    # Reemplazar nulos con la mediana
    columnas_mediana =['stays_in_weekend_nights','stays_in_week_nights']
    mediana = data[columnas_mediana].median()
    data[columnas_mediana] = data[columnas_mediana].fillna(mediana)
    # Verificar que los nulos en las columnas de mediana fueron reemplazados
    print("\nDespués de reemplazar nulos con la mediana:")
    print(data[columnas_mediana].isnull().sum())

    # Reemplazar nulos con fillna 0
    columnas_fillna_0=[
                        'arrival_date_year',
                        'arrival_date_day_of_month',
                        'adults',
                        'children',
                        'babies',
                        'previous_cancellations',
                        'previous_bookings_not_canceled',
                        'booking_changes',
                        'agent',
                        'days_in_waiting_list',
                        'required_car_parking_spaces',
                        'total_of_special_requests']
    
    data[columnas_fillna_0] = data[columnas_fillna_0].fillna(0)

   # Verificar que los nulos en las columnas de fillna(0) fueron reemplazados
    print("\nDespués de reemplazar nulos con 0:")
    print(data[columnas_fillna_0].isnull().sum())

    # Copiar el DataFrame 
    data_copia = data.copy()

    # Configurar y aplicar IterativeImputer directamente
    imputer_iterative = IterativeImputer(max_iter=20, random_state=42)
    data_copia['lead_time'] = imputer_iterative.fit_transform(data_copia[['lead_time']])

    # Verificar si se han eliminado los nulos
    print(f"Después del 'Iterative' tenemos: \n{data_copia['lead_time'].isnull().sum()} nulos")

    
    # Guardar el DataFrame en un archivo CSV
    data_copia.to_csv("finanzas-hotel-bookings_nonulls.csv", index=False)

    return data_copia

# Imputar valores nulos en variables numéricas
data_final = variables_numericas(df_categoricas)

Las columnas numéricas que tienen nulos son : 

Index(['lead_time', 'arrival_date_year', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'previous_cancellations', 'previous_bookings_not_canceled',
       'booking_changes', 'agent', 'days_in_waiting_list', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests'],
      dtype='object')
lead_time                         34.661002
arrival_date_year                 64.550490
arrival_date_day_of_month         34.780754
stays_in_weekend_nights           34.661002
stays_in_week_nights              34.661002
adults                            34.694904
children                          62.104584
babies                            34.693264
previous_cancellations            58.317339
previous_bookings_not_canceled    34.715683
booking_changes                   34.700372
agent                             43.650650
days_in_waiting_list              34.7156

In [13]:
data_final.isnull().sum()

hotel                                 0
is_canceled                           0
lead_time                             0
arrival_date_year                     0
arrival_date_month                    0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              0
babies                                0
meal                                  0
country                               0
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
agent                                 0
days_in_waiting_list                  0
customer_type                         0
adr                                   0


In [14]:
#duplicados del csv
data_final.duplicated().sum()

np.int64(70373)

In [15]:
# Lista de columnas a transformar de float64 a int64
columnas_a_convertir = [
    'lead_time',
    'arrival_date_year',
    'arrival_date_day_of_month',
    'stays_in_weekend_nights',
    'stays_in_week_nights',
    'adults',
    'children',
    'babies',
    'previous_cancellations',
    'previous_bookings_not_canceled',
    'booking_changes',
    'agent',
    'days_in_waiting_list',
    'required_car_parking_spaces',
    'total_of_special_requests'
]

# Convertir las columnas de float64 a int64
data_final[columnas_a_convertir] = data_final[columnas_a_convertir].astype('int64')


In [16]:
# Convertir valores de 1.0 a 'Yes' y 0.0 a 'No'
data_final['is_repeated_guest'] = data_final['is_repeated_guest'].apply(lambda x: 'Yes' if x == 1.0 else 'No')


In [17]:
# Convertir la columna de "Yes"/"No" a tipo categórico
data_final['is_repeated_guest'] = data_final['is_repeated_guest'].astype('category')


In [18]:
data_final['arrival_date_month'].unique()

array(['July', 'August', 'September', 'October', 'November', 'December',
       '1', 'January', 'February', '2', '3', 'March', 'April', 'May',
       'June', 'Unknown'], dtype=object)

In [19]:
#Cambiar numeros por meses
# Diccionario de mapeo de números a nombres de meses
meses = {
    '1': 'January',
    '2': 'February',
    '3': 'March',
    'January': 'January', 
    'February': 'February', 
    'March': 'March', 
    'April': 'April', 
    'May': 'May', 
    'June': 'June', 
    'July': 'July', 
    'August': 'August', 
    'September': 'September', 
    'October': 'October', 
    'November': 'November', 
    'December': 'December', 
    'Unknown': 'Unknown'
}

data_final['arrival_date_month'] = data_final['arrival_date_month'].map(meses)


In [20]:
data_final['arrival_date_month'].unique()

array(['July', 'August', 'September', 'October', 'November', 'December',
       'January', 'February', 'March', 'April', 'May', 'June', 'Unknown'],
      dtype=object)